# **Modelado: Volatilidad**

In [1]:
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
btc = pd.read_csv(r'https://raw.githubusercontent.com/TawnyVTC/Proyectos_UN/refs/heads/main/2025/Deep_Learning/Data/btc_1d_with_volatility_and_lags.csv')
btc['Open time'] = pd.to_datetime(btc['Open time'], format='%Y-%m-%d')


## **Split Temporal y Validación Cruzada**


En esta sección se replica la misma lógica empleada para la predicción del precio de cierre, pero enfocada en la volatilidad del Bitcoin. Primero, se generan variables retardadas de la volatilidad (lags de 7, 14, 21 y 28 días), que capturan la dependencia temporal del indicador y permiten al modelo aprender patrones históricos. Posteriormente, se crean los objetivos futuros correspondientes a los próximos siete días (t+1 a t+7), de modo que el modelo pueda realizar predicciones multihorizonte.

Las características de entrada incluyen tanto variables del precio (Close, LogReturn) como las medidas de volatilidad actual y retardada, buscando combinar información de nivel y dispersión. Finalmente, se eliminan los valores faltantes generados por los desplazamientos y se construyen las matrices de entrada (X) y salida (y) que alimentarán el flujo principal de entrenamiento, conformando el objeto timeSeries con el que se aplicarán los mismos experimentos de evaluación por número de lags.

In [3]:
# -----------------------
# 1. Crear lags de volatilidad
# -----------------------
for lag in [7, 14, 21, 28]:
    btc[f'Volatility_lag_{lag}'] = btc['Volatility'].shift(lag)

# -----------------------
# 2. Crear targets futuros (predicciones de volatilidad)
# -----------------------
for i in range(1, 8):  # t+1 a t+7
    btc[f'target_vol_t+{i}'] = btc['Volatility'].shift(-i)

# -----------------------
# 3. Definir features y targets
# -----------------------
features = [
    'Close', 'LogReturn', 'Volatility',
    'Volatility_lag_7', 'Volatility_lag_14', 'Volatility_lag_21', 'Volatility_lag_28'
]

targets = [f'target_vol_t+{i}' for i in range(1, 8)]

# -----------------------
# 4. Eliminar filas con NaN (por los shifts)
# -----------------------
btc = btc.dropna(subset=features + targets).reset_index(drop=True)

# -----------------------
# 5. Definir matrices de entrada y salida
# -----------------------
X = btc[features].values
y = btc[targets].values
dates = btc["Open time"].values

# -----------------------
# 6. Construcción del objeto timeSeries
# -----------------------
timeSeries = np.concatenate([X, y], axis=1)


In [4]:
from tsxv.splitTrainValTest import split_train_val_test_groupKFold
from sklearn.preprocessing import StandardScaler
import numpy as np

def _ensure_2d(arr):
    """
    Convierte el array a formato 2D.
    Si llega 1D → (n,1); si llega 3D → (n, timesteps*feats)
    """
    arr = np.asarray(arr)
    if arr.ndim == 1:
        return arr.reshape(-1, 1)
    if arr.ndim == 2:
        return arr
    if arr.ndim == 3:
        n, a, b = arr.shape
        return arr.reshape(n, a * b)
    raise ValueError(f"Array con ndim={arr.ndim} no soportado por esta función.")


def split_and_scale(timeSeries, n_steps_input=7, n_steps_forecast=7, n_steps_jump=1, target_col=2):
    X_list, y_list, Xcv_list, ycv_list, Xtest_list, ytest_list = split_train_val_test_groupKFold(
        timeSeries,
        n_steps_input,
        n_steps_forecast,
        n_steps_jump
    )

    X_train_scaled, X_val_scaled, X_test_scaled = [], [], []
    y_train_scaled, y_val_scaled, y_test_scaled = [], [], []
    scalers_x, scalers_y = [], []

    for fold in range(len(X_list)):
        X_train_raw = _ensure_2d(X_list[fold])
        X_val_raw   = _ensure_2d(Xcv_list[fold])
        X_test_raw  = _ensure_2d(Xtest_list[fold])

        # 👇 Aquí filtramos solo la variable objetivo (columna target)
        y_train_raw = _ensure_2d(y_list[fold])[:, target_col:target_col + n_steps_forecast]
        y_val_raw   = _ensure_2d(ycv_list[fold])[:, target_col:target_col + n_steps_forecast]
        y_test_raw  = _ensure_2d(ytest_list[fold])[:, target_col:target_col + n_steps_forecast]

        # Escaladores
        scaler_x = StandardScaler().fit(X_train_raw)
        scaler_y = StandardScaler().fit(y_train_raw)

        X_train_scaled.append(scaler_x.transform(X_train_raw))
        X_val_scaled.append(scaler_x.transform(X_val_raw))
        X_test_scaled.append(scaler_x.transform(X_test_raw))
        y_train_scaled.append(scaler_y.transform(y_train_raw))
        y_val_scaled.append(scaler_y.transform(y_val_raw))
        y_test_scaled.append(scaler_y.transform(y_test_raw))

        scalers_x.append(scaler_x)
        scalers_y.append(scaler_y)

        print(f"Fold {fold+1}: train {X_train_raw.shape} -> val {X_val_raw.shape} -> test {X_test_raw.shape}")

    return {
        'X_train': X_train_scaled,
        'X_val': X_val_scaled,
        'X_test': X_test_scaled,
        'y_train': y_train_scaled,
        'y_val': y_val_scaled,
        'y_test': y_test_scaled,
        'scalers_x': scalers_x,
        'scalers_y': scalers_y
    }


## **Modelado con Deep Learning**

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.stats.diagnostic import acorr_ljungbox
import pandas as pd
import numpy as np

In [6]:
def test_independencia_residuos(residuos, lags=10):
    resultado = acorr_ljungbox(residuos, lags=[lags], return_df=True)
    return resultado['lb_pvalue'].iloc[0]

def _calc_metrics_per_horizon(y_true, y_pred):
    """
    y_true, y_pred: arrays (n_samples, n_horizons)
    Devuelve DataFrame con métricas por horizonte.
    """
    n_outputs = y_true.shape[1]
    rows = []
    for h in range(n_outputs):
        yt = y_true[:, h]
        yp = y_pred[:, h]
        mae = mean_absolute_error(yt, yp)
        mse = mean_squared_error(yt, yp)
        rmse = np.sqrt(mse)
        # MAPE con epsilon para evitar división por cero
        mape = np.mean(np.abs((yt - yp) / (np.abs(yt) + 1e-8))) * 100
        rows.append({'Horizonte': h+1, 'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'MAPE': mape})
    df = pd.DataFrame(rows)
    return df

In [7]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

def entrenar_mlp_folds(data_dict, n_outputs=7, epochs=100, batch_size=32, verbose_fit=0, lag=None, save_dir='models'):
    """
    Entrena un modelo MLP para cada fold y guarda el mejor fold (según RMSE promedio en test).

    Parámetros:
    ------------
    data_dict : dict
        Diccionario con datos escalados (X_train, y_train, etc.)
    n_outputs : int
        Número de pasos de predicción (por defecto 7).
    epochs : int
        Épocas de entrenamiento.
    batch_size : int
        Tamaño del batch.
    verbose_fit : int
        Nivel de verbosidad del entrenamiento.
    lag : int, opcional
        Número de lag, usado para nombrar la carpeta de guardado.
    save_dir : str
        Carpeta base donde se guardarán los modelos.

    Retorna:
    --------
    resultados : list
        Lista con resultados por fold (predicciones, métricas, etc.)
    tablas_folds : list
        Lista de dataframes con métricas por horizonte.
    best_fold : int
        Índice del mejor fold (basado en RMSE promedio test).
    """

    resultados = []
    tablas_folds = []

    for fold in range(len(data_dict['X_train'])):
        print(f"\n===== Fold {fold+1}/{len(data_dict['X_train'])} =====")

        X_train, X_val, X_test = data_dict['X_train'][fold], data_dict['X_val'][fold], data_dict['X_test'][fold]
        y_train, y_val, y_test = data_dict['y_train'][fold], data_dict['y_val'][fold], data_dict['y_test'][fold]
        scaler_y = data_dict['scalers_y'][fold]

        # --- Modelo ---
        model = Sequential([
            Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
            Dropout(0.2),
            Dense(32, activation='relu'),
            Dense(n_outputs)
        ])
        model.compile(optimizer='adam', loss='mse')

        es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        model.fit(X_train, y_train, validation_data=(X_val, y_val),
                  epochs=epochs, batch_size=batch_size, verbose=verbose_fit, callbacks=[es])

        # --- Predicciones desescaladas ---
        yhat_train = scaler_y.inverse_transform(model.predict(X_train))
        yhat_val = scaler_y.inverse_transform(model.predict(X_val))
        yhat_test = scaler_y.inverse_transform(model.predict(X_test))
        ytrain_real = scaler_y.inverse_transform(y_train)
        yval_real = scaler_y.inverse_transform(y_val)
        ytest_real = scaler_y.inverse_transform(y_test)

        # --- Métricas ---
        df_metrics = _calc_metrics_per_horizon(ytest_real, yhat_test)
        promedio = df_metrics.mean(numeric_only=True).to_dict()
        promedio['Horizonte'] = 'Promedio'
        df_metrics = pd.concat([df_metrics, pd.DataFrame([promedio])], ignore_index=True)

        # p-value BDS
        resid_h1 = ytest_real[:, 0] - yhat_test[:, 0]
        try:
            pval = test_independencia_residuos(resid_h1, lags=10)
        except Exception:
            pval = np.nan

        bds_col = [np.nan] * len(df_metrics)
        if len(df_metrics) > 0:
            bds_col[0] = pval
        df_metrics['BDS_pvalue_h1'] = bds_col

        rmse_prom = df_metrics.loc[df_metrics['Horizonte'] == 'Promedio', 'RMSE'].values[0]
        print(f"Fold {fold+1} | RMSE prom: {rmse_prom:.4f} | pval(h1): {pval}")

        resultados.append({
            'fold': fold+1,
            'rmse_test_prom': rmse_prom,
            'pval_h1': pval,
            'y_train_real': ytrain_real, 'y_train_pred': yhat_train,
            'y_val_real': yval_real, 'y_val_pred': yhat_val,
            'y_test_real': ytest_real, 'y_test_pred': yhat_test,
            'df_metrics': df_metrics,
            'model': model
        })

        tablas_folds.append(df_metrics)

    # --- Seleccionar mejor fold ---
    rmse_vals = [r['rmse_test_prom'] for r in resultados]
    best_fold = int(np.argmin(rmse_vals))
    best_model = resultados[best_fold]['model']
    print(f"\n✅ Mejor fold: {best_fold+1} con RMSE promedio {rmse_vals[best_fold]:.4f}")

    # --- Guardar modelo ---
    if lag is not None:
        model_dir = os.path.join(save_dir, f"lag_{lag}")
        os.makedirs(model_dir, exist_ok=True)
        model_path = os.path.join(model_dir, f"mejor_fold_lag_{lag}.keras")
        best_model.save(model_path)
        print(f"💾 Modelo guardado en: {model_path}")

    return resultados, tablas_folds


In [8]:
# ------------------------------------------------------------------
# Funciones para agregar resultados por lag y crear tablas resumen
# ------------------------------------------------------------------

def resumen_por_lag(tablas_folds):
    """
    tablas_folds: lista de dataframes (uno por fold), cada df tiene las filas por horizonte + 'Promedio'.
    Devuelve df resumen por horizonte con mean y std entre folds.
    """
    # extraer solo filas de horizonte 1..H (no 'Promedio') para hacer stats por horizonte
    # asumimos que todos los df tienen la misma cantidad de horizontes antes de la fila Promedio
    list_horizon_dfs = []
    for df in tablas_folds:
        # seleccionar solo filas cuyo 'Horizonte' no sea 'Promedio'
        df_h = df[df['Horizonte'] != 'Promedio'].copy()
        df_h = df_h.reset_index(drop=True)
        list_horizon_dfs.append(df_h)

    # concatenar en multiindex: fold, horizonte
    concat = pd.concat(list_horizon_dfs, keys=range(1, len(list_horizon_dfs)+1), names=['fold','row'])
    # Queremos agrupar por horizonte y calcular mean/std de métricas
    metrics = ['MAPE','MAE','RMSE','MSE']
    summary_rows = []
    for h in sorted(concat['Horizonte'].unique(), key=lambda x: int(x)):
        sel = concat[concat['Horizonte']==h]
        row = {'Horizonte': int(h)}
        for m in metrics:
            row[f'{m}_mean'] = sel[m].mean()
            row[f'{m}_std'] = sel[m].std()
        # p-value mean across folds: buscar pval en fold df (está en la primera fila BDS_pvalue_h1)
        # coletar pvals
        pvals = []
        for df in tablas_folds:
            pv = df['BDS_pvalue_h1'].iloc[0]
            if not np.isnan(pv):
                pvals.append(pv)
        row['BDS_pvalue_h1_mean'] = np.mean(pvals) if len(pvals)>0 else np.nan
        row['BDS_pvalue_h1_std'] = np.std(pvals) if len(pvals)>0 else np.nan

        summary_rows.append(row)

    df_summary = pd.DataFrame(summary_rows)
    return df_summary


In [9]:
# ------------------------------------------------------------------
# Función principal: evalúa lista de lags y devuelve todo lo necesario
# ------------------------------------------------------------------

def evaluar_y_reportar(timeSeries, lags_list=[7,14,21,28], n_outputs=7,
                        epochs=80, batch_size=32, verbose_fit=0):
    """
    Ejecuta todo el flujo para cada n_lag:
    - split + scale
    - entrenar por folds
    - guarda tablas por fold y resumen por lag (mean + std)
    - devuelve estructura con todo para plotting y tablas
    """
    todos_lags = {}
    comparativa_resumen = []

    for n_lag in lags_list:
        print(f"\n=== Procesando n_lag = {n_lag} ===")
        data_dict = split_and_scale(timeSeries, n_steps_input=n_lag, n_steps_forecast=n_outputs)
        resultados, tablas_folds = entrenar_mlp_folds(data_dict, n_outputs=n_outputs,
                                                     epochs=epochs, batch_size=batch_size,
                                                     verbose_fit=verbose_fit)

        # df concatenado de métricas (todos los folds)
        df_metricas_comb = pd.concat([t.reset_index(drop=True) for t in tablas_folds], keys=range(1, len(tablas_folds)+1),
                                      names=['fold','row']).reset_index()
        # resumen por horizonte (mean, std)
        df_summary_h = resumen_por_lag(tablas_folds)

        # resumen global (media de RMSE, MAE, MAPE, MSE y pval promedio)
        rmse_prom = df_metricas_comb.groupby('Horizonte')['RMSE'].mean().mean()  # promedio de horizontes
        mae_prom  = df_metricas_comb.groupby('Horizonte')['MAE'].mean().mean()
        mape_prom = df_metricas_comb.groupby('Horizonte')['MAPE'].mean().mean()
        mse_prom  = df_metricas_comb.groupby('Horizonte')['MSE'].mean().mean()
        pvals = df_metricas_comb['BDS_pvalue_h1'].dropna()
        pval_prom = pvals.mean() if len(pvals)>0 else np.nan

        comparativa_resumen.append({
            'n_lag': n_lag,
            'RMSE_prom': rmse_prom,
            'RMSE_std': df_metricas_comb.groupby('Horizonte')['RMSE'].mean().std(),
            'MAE_prom': mae_prom,
            'MAPE_prom': mape_prom,
            'MSE_prom': mse_prom,
            'BDS_pvalue_h1_prom': pval_prom
        })

        todos_lags[n_lag] = {
            'resultados': resultados,         # lista de dict por fold (incluye df_metrics por fold)
            'tablas_folds': tablas_folds,     # lista de dataframes por fold
            'df_metricas_comb': df_metricas_comb,
            'df_summary_h': df_summary_h
        }

    df_comparativa = pd.DataFrame(comparativa_resumen).sort_values('n_lag').reset_index(drop=True)
    return todos_lags, df_comparativa

In [10]:
# ------------------------------------------------------------------
# Funciones de plotting
# ------------------------------------------------------------------

def escoger_folds_por_rmse(resultados):
    """
    Recibe lista de resultados (cada uno con 'rmse_test_prom').
    Devuelve indices best (min rmse), worst (max rmse) y median (por valor).
    """
    rmses = [r['rmse_test_prom'] for r in resultados]
    order = np.argsort(rmses)
    best_idx = int(order[0])
    worst_idx = int(order[-1])
    median_pos = int(len(order)//2)
    median_idx = int(order[median_pos])
    return best_idx, median_idx, worst_idx


def plot_series_fold(resultados_fold, fold_idx, n_steps_input, title_prefix=""):
    """
    Grafica las series reales y predichas para un fold específico.
    """
    y_train_real = resultados_fold['y_train_real']
    y_train_pred = resultados_fold['y_train_pred']
    y_val_real   = resultados_fold['y_val_real']
    y_val_pred   = resultados_fold['y_val_pred']
    y_test_real  = resultados_fold['y_test_real']
    y_test_pred  = resultados_fold['y_test_pred']

    fig, ax = plt.subplots(figsize=(10, 5))

    # Entrenamiento
    ax.plot(y_train_real[:, 0], label='Train Real', color='tab:blue', alpha=0.6)
    ax.plot(y_train_pred[:, 0], label='Train Predicho', color='tab:cyan', linestyle='--', alpha=0.8)

    # Validación
    ax.plot(range(len(y_train_real), len(y_train_real)+len(y_val_real)),
             y_val_real[:, 0], label='Val Real', color='tab:orange', alpha=0.6)
    ax.plot(range(len(y_train_real), len(y_train_real)+len(y_val_pred)),
             y_val_pred[:, 0], label='Val Predicho', color='tab:red', linestyle='--', alpha=0.8)

    # Test
    ax.plot(range(len(y_train_real)+len(y_val_real),
                  len(y_train_real)+len(y_val_real)+len(y_test_real)),
             y_test_real[:, 0], label='Test Real', color='tab:green', alpha=0.6)
    ax.plot(range(len(y_train_real)+len(y_val_real),
                  len(y_train_real)+len(y_val_real)+len(y_test_pred)),
             y_test_pred[:, 0], label='Test Predicho', color='tab:purple', linestyle='--', alpha=0.8)

    ax.set_title(f"{title_prefix} | Fold {fold_idx+1} | Ventana = {n_steps_input}")
    ax.set_xlabel("Tiempo")
    ax.set_ylabel("Valor (Volatilidad o Precio)")
    ax.legend()
    ax.grid(alpha=0.3)

    fig.tight_layout()
    return fig


def plot_rmse_bars(resultados, n_steps_input):
    """
    Grafica barras con el RMSE promedio por fold.
    """
    import numpy as np
    import matplotlib.pyplot as plt

    rmses = [r['rmse_test_prom'] for r in resultados]
    folds = np.arange(1, len(rmses)+1)

    fig, ax = plt.subplots(figsize=(8, 4))
    ax.bar(folds, rmses, color='skyblue', edgecolor='k')
    ax.set_title(f"RMSE por Fold | Ventana = {n_steps_input}")
    ax.set_xlabel("Fold")
    ax.set_ylabel("RMSE promedio (test)")
    ax.grid(axis='y', alpha=0.3)

    for i, v in enumerate(rmses):
        ax.text(folds[i], v, f"{v:.3f}", ha='center', va='bottom', fontsize=8)

    fig.tight_layout()
    return fig


def plot_rmse_promedio_por_horizonte(df_summary_h, n_steps_input):
    """
    Muestra el RMSE promedio (y std si existe) por horizonte.
    """
    fig, ax = plt.subplots(figsize=(8, 4))

    ax.plot(df_summary_h.index, df_summary_h['RMSE_mean'], marker='o', label='RMSE promedio')

    if 'RMSE_std' in df_summary_h.columns:
        ax.fill_between(df_summary_h.index,
                        df_summary_h['RMSE_mean'] - df_summary_h['RMSE_std'],
                        df_summary_h['RMSE_mean'] + df_summary_h['RMSE_std'],
                        color='blue', alpha=0.2, label='Desviación estándar')

    ax.set_title(f"RMSE promedio por horizonte | Ventana = {n_steps_input}")
    ax.set_xlabel("Horizonte de predicción (h)")
    ax.set_ylabel("RMSE")
    ax.legend()
    ax.grid(alpha=0.3)

    fig.tight_layout()
    return fig


In [11]:
# 1) Ejecutar evaluación completa
todos_lags, df_comparativa = evaluar_y_reportar(timeSeries, lags_list=[7,14,21,28],
                                                n_outputs=7, epochs=80, batch_size=32,
                                                verbose_fit=0)


=== Procesando n_lag = 7 ===
Fold 1: train (325, 98) -> val (108, 98) -> test (108, 98)
Fold 2: train (324, 98) -> val (108, 98) -> test (108, 98)
Fold 3: train (323, 98) -> val (108, 98) -> test (108, 98)
Fold 4: train (322, 98) -> val (108, 98) -> test (108, 98)
Fold 5: train (324, 98) -> val (108, 98) -> test (108, 98)

===== Fold 1/5 =====
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Fold 1 | RMSE prom: 0.0693 | pval(h1): 0.8915469129181351

===== Fold 2/5 =====
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Fold 2 | RMSE prom: 0.0698 | pval(h1): 0.005072758465731627

===== Fold 3/5 =====
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Fold 3 | RMSE prom: 0.0605 | pval(h1): 0.142210402547321

===== Fold 4/5 =====
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━

En la etapa de modelado de volatilidad, se evaluó el desempeño del modelo bajo diferentes configuraciones de retardos temporales (7, 14, 21 y 28 días). Los resultados muestran un comportamiento estable para horizontes cortos, con valores promedio de RMSE entre 0.06 y 0.07 para los modelos con 7 y 14 lags, indicando una adecuada capacidad del modelo para capturar la variabilidad de corto plazo en la serie.

A medida que se incrementa la ventana de rezagos (21 y 28 días), se observa un aumento gradual del error (RMSE entre 0.09 y 0.11), lo que sugiere una pérdida de precisión debido a la menor cantidad de datos efectivos para el entrenamiento y la posible dilución de patrones recientes. Los valores del p-valor del test BDS en la mayoría de los pliegues superan el umbral de significancia (p > 0.05), lo que sugiere que los residuales no presentan dependencias no lineales relevantes, respaldando la validez del modelo en términos de independencia temporal.

En general, el modelo logra una representación coherente del comportamiento dinámico de la volatilidad, especialmente en horizontes cortos, donde la memoria reciente del mercado parece ser más informativa para la predicción.

### *Tabla de Resumen Comparativo por Lags*

In [12]:
# 2) Ver tabla resumen comparativa de los 4 lags:
df_comparativa.to_csv(r'C:\DeepLearning\DL_Proyecto_2\data\volatilidad\tablas\resumen_comparativo.csv', index=False)
df_comparativa

,n_lag,RMSE_prom,RMSE_std,MAE_prom,MAPE_prom,MSE_prom,BDS_pvalue_h1_prom
0,7,0.062286,0.009220,0.038070,6.741061,0.004175,0.433037
1,14,0.062680,0.015837,0.044752,7.859136,0.004328,0.505995
2,21,0.106995,0.033413,0.068251,11.930163,0.013246,0.756420
3,28,0.091503,0.015232,0.066055,11.799854,0.008978,0.583842


Los resultados del modelo para la predicción de volatilidad del Bitcoin muestran un desempeño estable y coherente a través de las distintas configuraciones de retardos. Los menores errores promedio se observan con lags de 7 y 14 días, con valores de RMSE ≈ 0.065 y MAE ≈ 0.04–0.045, lo que evidencia una buena capacidad del modelo para capturar la dinámica de corto plazo.

En cambio, al incrementar la memoria temporal (lags de 21 y 28 días), los errores aumentan ligeramente (RMSE entre 0.09 y 0.11), lo que sugiere que incluir retardos más largos introduce ruido o patrones menos relevantes para la predicción inmediata.

El MAPE, que se mantiene entre 7% y 12%, refuerza la estabilidad general del modelo en términos relativos.
Finalmente, los valores del p-valor del test BDS (entre 0.52 y 0.71) superan el umbral de significancia (0.05), indicando que los residuales no presentan dependencias no lineales, por lo que el modelo logra capturar adecuadamente la estructura temporal de la volatilidad

In [13]:
import os

def resultados_lag_vol(n_lag):
    # 3) Para un lag específico:
    info = todos_lags[n_lag]
    resultados = info['resultados']   # lista por fold
    tablas_folds = info['tablas_folds']
    df_metricas_comb = info['df_metricas_comb']
    df_summary_h = info['df_summary_h']

    # Crear carpetas si no existen
    base_dir = fr'C:\DeepLearning\DL_Proyecto_2\data\volatilidad'
    tablas_dir = os.path.join(base_dir, 'tablas', f'lag_{n_lag}')
    figs_dir = os.path.join(base_dir, 'figs', f'lag_{n_lag}')
    os.makedirs(tablas_dir, exist_ok=True)
    os.makedirs(figs_dir, exist_ok=True)

    # 4) Guardar tablas por fold (test)
    for i, df in enumerate(tablas_folds):
        print(f"\n--- Fold {i+1} ---")
        df_path = os.path.join(tablas_dir, f'fold_{i+1}_lag_{n_lag}.csv')
        df.to_csv(df_path, index=False)
        print(df)

    # 5) Seleccionar best/median/worst y plotear
    best, med, worst = escoger_folds_por_rmse(resultados)

    # Mejor
    fig = plot_series_fold(resultados[best], best, n_steps_input=n_lag, title_prefix="Mejor (RMSE)")
    fig.savefig(os.path.join(figs_dir, f'mejor_rmse_lag_{n_lag}.png'), bbox_inches='tight')
    plt.close()

    # Mediano
    fig = plot_series_fold(resultados[med], med, n_steps_input=n_lag, title_prefix="Mediano (RMSE)")
    fig.savefig(os.path.join(figs_dir, f'mediano_rmse_lag_{n_lag}.png'), bbox_inches='tight')
    plt.close(fig)

    # Peor
    fig = plot_series_fold(resultados[worst], worst, n_steps_input=n_lag, title_prefix="Peor (RMSE)")
    fig.savefig(os.path.join(figs_dir, f'peor_rmse_lag_{n_lag}.png'), bbox_inches='tight')
    plt.close(fig)

    # 6) Gráficas adicionales
    fig = plot_rmse_bars(resultados, n_steps_input=n_lag)
    fig.savefig(os.path.join(figs_dir, f'rmse_por_fold_lag_{n_lag}.png'), bbox_inches='tight')
    plt.close(fig)

    fig = plot_rmse_promedio_por_horizonte(df_summary_h, n_steps_input=n_lag)
    fig.savefig(os.path.join(figs_dir, f'rmse_promedio_por_horizonte_lag_{n_lag}.png'), bbox_inches='tight')
    plt.close(fig)

    print(f"\n✅ Resultados y gráficas guardados en: {figs_dir}")


### *Tablas de Metricas: Resumen por Horizonte y por Fold*

In [14]:
resultados_lag_vol(7)


--- Fold 1 ---
  Horizonte       MAE       MSE      RMSE      MAPE  BDS_pvalue_h1
0         1  0.037487  0.010490  0.102420  6.743446       0.891547
1         2  0.036673  0.002706  0.052023  6.757439            NaN
2         3  0.036455  0.003305  0.057486  6.473721            NaN
3         4  0.043625  0.006507  0.080668  6.620333            NaN
4         5  0.039824  0.003210  0.056658  7.597544            NaN
5         6  0.031414  0.004778  0.069124  4.358839            NaN
6         7  0.029271  0.004475  0.066892  3.993403            NaN
7  Promedio  0.036393  0.005067  0.069324  6.077818            NaN

--- Fold 2 ---
  Horizonte       MAE       MSE      RMSE      MAPE  BDS_pvalue_h1
0         1  0.041829  0.005825  0.076321  8.499314       0.005073
1         2  0.040635  0.003032  0.055062  6.992410            NaN
2         3  0.045862  0.004675  0.068376  8.276422            NaN
3         4  0.052068  0.006957  0.083410  8.312861            NaN
4         5  0.043751  0.00325

In [15]:
resultados_lag_vol(14)


--- Fold 1 ---
  Horizonte       MAE       MSE      RMSE       MAPE  BDS_pvalue_h1
0         1  0.034595  0.002171  0.046596   6.316844       0.663502
1         2  0.028874  0.002229  0.047214   4.741901            NaN
2         3  0.040861  0.004049  0.063635   6.150395            NaN
3         4  0.053933  0.007572  0.087016   8.189806            NaN
4         5  0.059110  0.007552  0.086904  10.396623            NaN
5         6  0.030067  0.001559  0.039479   5.664456            NaN
6         7  0.033609  0.002626  0.051243   6.360378            NaN
7  Promedio  0.040150  0.003965  0.060298   6.831486            NaN

--- Fold 2 ---
  Horizonte       MAE       MSE      RMSE       MAPE  BDS_pvalue_h1
0         1  0.040478  0.002613  0.051121   7.089398        0.73095
1         2  0.041996  0.003173  0.056325   6.879242            NaN
2         3  0.048059  0.003820  0.061810   8.361545            NaN
3         4  0.065098  0.009452  0.097220   9.891707            NaN
4         5  0.0

In [16]:
resultados_lag_vol(21)


--- Fold 1 ---
  Horizonte       MAE       MSE      RMSE       MAPE  BDS_pvalue_h1
0         1  0.086956  0.029092  0.170563  13.287351       0.767607
1         2  0.073124  0.009465  0.097287  13.410948            NaN
2         3  0.052059  0.004170  0.064573   9.095380            NaN
3         4  0.069181  0.007379  0.085901  12.541135            NaN
4         5  0.056203  0.006236  0.078971  11.180129            NaN
5         6  0.089295  0.043616  0.208844  11.705412            NaN
6         7  0.087969  0.030292  0.174047  13.779760            NaN
7  Promedio  0.073541  0.018607  0.125741  12.142874            NaN

--- Fold 2 ---
  Horizonte       MAE       MSE      RMSE       MAPE  BDS_pvalue_h1
0         1  0.091547  0.032640  0.180665  13.819671        0.66901
1         2  0.069776  0.011543  0.107436  13.614970            NaN
2         3  0.079442  0.009391  0.096909  15.100938            NaN
3         4  0.065571  0.006948  0.083357  13.854645            NaN
4         5  0.0

In [17]:
resultados_lag_vol(28)


--- Fold 1 ---
  Horizonte       MAE       MSE      RMSE       MAPE  BDS_pvalue_h1
0         1  0.054831  0.004753  0.068945   9.442787       0.915498
1         2  0.068354  0.008773  0.093664  10.821783            NaN
2         3  0.061655  0.007321  0.085561  13.322803            NaN
3         4  0.051690  0.005566  0.074604   8.857666            NaN
4         5  0.076874  0.010780  0.103828  15.025700            NaN
5         6  0.059571  0.006328  0.079550  11.420390            NaN
6         7  0.071440  0.007922  0.089008  12.256295            NaN
7  Promedio  0.063488  0.007349  0.085023  11.592489            NaN

--- Fold 2 ---
  Horizonte       MAE       MSE      RMSE       MAPE  BDS_pvalue_h1
0         1  0.055755  0.007502  0.086614   9.510814       0.144472
1         2  0.051335  0.004167  0.064553   8.385306            NaN
2         3  0.058653  0.005677  0.075347  11.802276            NaN
3         4  0.048319  0.004614  0.067930   9.176949            NaN
4         5  0.0